In [1]:
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [3]:
import time

In [4]:
#You need to have driver corresponding to your Google Chrome browser or update browser
from selenium.webdriver.common.by import By

In [5]:
#CS  - CS 
#EC  - ECE
#CI  - AI
#CSE - CS+AI
RESULT_FOR="CSE"
YEAR="21"

In [6]:
usns=[]
# Change Range According to Number of students
if(RESULT_FOR!="CSE"):
    #Students USN 
    for i in range(1,366):
        usn='01FE'+YEAR+'B'+RESULT_FOR+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)
    
else:
    #CS + AI USNS
    for i in range(2,367):
        usn='01FE'+YEAR+'BCS'+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)
    for i in range(1,63):
        usn='01FE'+YEAR+'BCI'+((3-len(str(i)))*'0')+str(i)
        usns.append(usn)

### Run the cell below and wait until Google Chrome opens 

In [7]:
url="https://eresults.kletech.ac.in/index.php"

#This is the path name of chrome driver on my PC
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
driver.maximize_window()

### When website Opens then change the captcha below and run the cell

### My college website has a glitch that its captcha doesn't change if we navigate back on the website

In [8]:
#put the captcha as shown in website and run this cell
captcha="W9393" #this was the current captcha when I was using

In [9]:
#For each USN I am scrapping the details
result={}
result['NAME']=[None]*len(usns)
result['USN']=['None']*len(usns)
result['SGPA']=[None]*len(usns)
result['CGPA']=[None]*len(usns)

for number in range(len(usns)):
    username=usns[number]
    driver.find_element(by=By.NAME,value="usn").send_keys(username)
    driver.find_element(by=By.NAME,value="osolCatchaTxt").send_keys(captcha)

    # driver.find_element_by_formaction("index.php?option=com_examresult&task=getResult").click()
    driver.find_element(By.XPATH, '//button[text()="Result"]').click()
        
    try:
        name=driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-body stu-data stu-data1")]')
        table=driver.find_element(By.XPATH,'//table[contains(@class,"uk-table uk-table-striped res-table")]/tbody')
        sgpa=driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec3")]')
        cgpa=driver.find_element(By.XPATH,'//div[contains(@class,"uk-card uk-card-default uk-card-body credits-sec4")]')
        sub=table.text.split('\n')
        grades=""
        print()
        print('=======================')
        print('Name:',name.text)
        print('USN:',username)
        print('=======SUBJECTS========')
        for i in range(len(sub)):
            t=sub[i].split(" ")
            title=" ".join(t[1:-3])
            grades=t[-1]
            print(title,':',grades)
            if(result.get(title)==None):
                result[title]=[None]*len(usns)
            result[title][number]=grades

        #print(grades)
        print('=======SUBJECTS========')
        print('SGPA:',sgpa.text[5:])
        print('CGPA',cgpa.text[5:])
        print('=======================')    
        print()
        result['NAME'][number]=name.text
        result['USN'][number]=username
        result['SGPA'][number]=sgpa.text[5:]
        result['CGPA'][number]=cgpa.text[5:]
        #file.write('\n'+(username)+','+(name.text)+','+(sgpa.text[5:])+','+(cgpa.text[5:])+','+grades)
    except:
        print("Error in fetching values for USN",username)
        
    driver.back() #navigating back by clicking back button

    driver.find_element(by=By.NAME,value="usn").clear() #clearing the already filled detail
    driver.find_element(by=By.NAME,value="osolCatchaTxt").clear() #Clearing the CAPTCHA field as it will be Entered again

print('==========')
print("Completed")
print('==========')
#print(result)


Name: AMOGH M MURAGODMATH
USN: 01FE21BCS002
=======SUBJECTS========
Mini Project : S
System Software : B
Computer Networks-1 : A
System Software Lab : A
Web Technologies Lab : B
Software Engineering : B
Machine Learning : S
Arithmetical Thinking and Analytical Reasoning : C
Natural Language processing with Neural Network models : B
=======SUBJECTS========
SGPA: 8.80
CGPA 8.25


Name: SAISAMARTH S UDIKERI
USN: 01FE21BCS003
=======SUBJECTS========
Mini Project : A
System Software : B
Computer Networks-1 : D
System Software Lab : A
Web Technologies Lab : B
Software Engineering : B
Machine Learning : B
Arithmetical Thinking and Analytical Reasoning : D
Natural Language processing with Neural Network models : A
=======SUBJECTS========
SGPA: 7.94
CGPA 6.62


Name: JAGADEESH A KALYAN SHETTI
USN: 01FE21BCS004
=======SUBJECTS========
Mini Project : S
System Software : D
Computer Networks-1 : D
System Software Lab : C
Web Technologies Lab : C
Software Engineering : E
Machine Learning : B
Intern

In [10]:
import pandas as pd
df=pd.DataFrame.from_dict(result)
df = df[df['USN'].str.contains('None') == False]
df.to_csv('result_B'+RESULT_FOR+'_'+YEAR+'_V.csv',index=False)